# Introduction to Retrieval Augmented Generation with S&P 500 news

In this notebook, you will explore how to build a simple Retrieval-Augmented Generation (RAG) pipeline using financial news articles from S&P 500 companies.

We'll start by vectorizing text data, creating a vector store using FAISS, and integrating it with OpenAI's GPT models to answer questions using retrieved information.

This workflow emulates real-world systems in finance where natural language data (news, filings, analyst reports) are used to support decision-making.

# 📌 Objectives

By the end of this notebook, students will be able to:

1. **Perform Semantic Search with Metadata Filtering:**
   - Query the provided FAISS vector store to retrieve relevant financial news articles based on natural language questions.
   - Apply optional filters using metadata such as ticker or publication date to refine search results.

2. **Enrich Data with Company Metadata:**
   - Use the `yfinance` library to retrieve company-level metadata (company name, sector, industry) for tickers in the dataset.
   - Integrate this metadata to support enhanced filtering and analysis of news data.

3. **Build a Retrieval-Augmented Generation (RAG) Pipeline:**
   - Combine retrieved news snippets as context to generate answers using OpenAI’s GPT models.
   - Construct effective prompts that guide the language model to provide concise, context-aware responses.

4. **Evaluate and Analyze RAG Outputs:**
   - Review generated answers alongside the supporting news excerpts.
   - Reflect on the strengths and limitations of the simple RAG pipeline and consider potential improvements, such as adding more filters or refining retrieval strategies.

5. **Incorporate Financial Metadata into Retrieval Context:**
   - Enrich retrieved news snippets with key financial metadata including ticker, company name, sector, and industry.
   - Format prompts that combine both text excerpts and metadata to provide richer context to the language model.

6. **Generate Context-Aware Answers Using OpenAI Models:**
   - Construct and send prompts to an LLM that leverage both news content and metadata to produce concise, informed financial analysis.

7. **Compare Answers With and Without Metadata:**
   - Evaluate the impact of including financial metadata on answer quality using criteria such as clarity, detail, accuracy, and contextual relevance.
   - Summarize findings to reflect on the role of metadata in improving retrieval-augmented generation.

## Install and Import important librairies

First, we install and import the necessary libraries for:
- Text embedding generation (sentence-transformers)
- Efficient similarity search (faiss)
- Data manipulation (pandas, numpy)
- Visualization (matplotlib)

> ℹ️ FAISS uses inner product for cosine similarity by normalizing vectors.

In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import matplotlib.pyplot as plt
import faiss
from dotenv import load_dotenv
import os
import yfinance as yf
import time
import httpx
import textwrap

## Load news data
We load a CSV file of financial news, focusing on TITLE and SUMMARY, along with metadata like TICKER and PUBLICATION_DATE.
These will be embedded into vectors and used for semantic retrieval.

In [2]:
K = 25

In [3]:
df_news = pd.read_csv('./data/df_news.csv')
df_news['PUBLICATION_DATE'] = pd.to_datetime(df_news['PUBLICATION_DATE']).dt.date
display(df_news)

,TICKER,TITLE,SUMMARY,PUBLICATION_DATE,PROVIDER,URL
0,MMM,2 Dow Jones Stocks with Promising Prospects an...,The Dow Jones (^DJI) is made up of 30 of the m...,2025-05-29,StockStory,https://finance.yahoo.com/news/2-dow-jones-sto...
1,MMM,3 S&P 500 Stocks Skating on Thin Ice,The S&P 500 (^GSPC) is often seen as a benchma...,2025-05-27,StockStory,https://finance.yahoo.com/news/3-p-500-stocks-...
2,MMM,3M Rises 15.8% YTD: Should You Buy the Stock N...,"MMM is making strides in the aerospace, indust...",2025-05-22,Zacks,https://finance.yahoo.com/news/3m-rises-15-8-y...
3,MMM,Q1 Earnings Roundup: 3M (NYSE:MMM) And The Res...,Quarterly earnings results are a good time to ...,2025-05-22,StockStory,https://finance.yahoo.com/news/q1-earnings-rou...
4,MMM,3 Cash-Producing Stocks with Questionable Fund...,While strong cash flow is a key indicator of s...,2025-05-19,StockStory,https://finance.yahoo.com/news/3-cash-producin...
...,...,...,...,...,...,...
4866,ZTS,2 Dividend Stocks to Buy With $500 and Hold Fo...,Zoetis is a leading animal health company with...,2025-05-23,Motley Fool,https://www.fool.com/investing/2025/05/23/2-di...
4867,ZTS,Zoetis (NYSE:ZTS) Declares US$0.50 Dividend Pe...,Zoetis (NYSE:ZTS) recently affirmed a dividend...,2025-05-22,Simply Wall St.,https://finance.yahoo.com/news/zoetis-nyse-zts...
4868,ZTS,Jim Cramer on Zoetis (ZTS): “It Does Seem to B...,We recently published a list of Jim Cramer Tal...,2025-05-21,Insider Monkey,https://finance.yahoo.com/news/jim-cramer-zoet...
4869,ZTS,Zoetis (ZTS) Upgraded to Buy: Here's Why,Zoetis (ZTS) might move higher on growing opti...,2025-05-21,Zacks,https://finance.yahoo.com/news/zoetis-zts-upgr...


In [4]:
df_news['EMBEDDED_TEXT'] = df_news['TITLE'] + ' : ' + df_news['SUMMARY']

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

## Implement FAISS vector store
We:
- Use a pre-trained sentence transformer (all-MiniLM-L6-v2) to embed documents.
- Normalize vectors to use cosine similarity.
- Create a FAISS index and implement a basic search function.

This will allow us to retrieve relevant news snippets given a natural language question.
 

In [6]:
# Load model and compute embeddings
text_embeddings = model.encode(df_news['EMBEDDED_TEXT'].tolist(), convert_to_numpy=True)

# Normalize embeddings to use cosine similarity (via inner product in FAISS)
text_embeddings = text_embeddings / np.linalg.norm(text_embeddings, axis=1, keepdims=True)

# Prepare metadata
documents = df_news['EMBEDDED_TEXT'].tolist()
metadata = [
    {
        'PUBLICATION_DATE': row['PUBLICATION_DATE'], 
        'TICKER': row['TICKER'], 
        'PROVIDER': row['PROVIDER']
    }
    for _, row in df_news.iterrows()
]

In [7]:
embedding_dim = text_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(embedding_dim)  # Cosine similarity via inner product
faiss_index.add(text_embeddings)

In [8]:
class FaissVectorStore:
    def __init__(self, model, index, embeddings, documents, metadata):
        self.model = model
        self.index = index
        self.embeddings = embeddings
        self.documents = documents
        self.metadata = metadata

    def search(self, query, k=5, metadata_filter=None):
        query_embedding = self.model.encode([query])
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        if metadata_filter:
            filtered_indices = [i for i, meta in enumerate(self.metadata) if metadata_filter(meta)]
            if not filtered_indices:
                return []
            filtered_embeddings = self.embeddings[filtered_indices]
            temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
            temp_index.add(filtered_embeddings)
            D, I = temp_index.search(query_embedding, k)
            indices = [filtered_indices[i] for i in I[0]]
        else:
            D, I = self.index.search(query_embedding, k)
            indices = I[0]
            D = D[0]

        results = []
        for idx, sim in zip(indices, D):
            results.append((self.documents[idx], self.metadata[idx], float(sim)))
        return results

In [9]:
# Create FAISS-based store
faiss_store = FaissVectorStore(
    model=model,
    index=faiss_index,
    embeddings=text_embeddings,
    documents=documents,
    metadata=metadata
)

### Setup OpenAI Client

👉 **Instructions**:
- Import the `OpenAI` client from the `openai` Python library.
- You will need an **OpenAI API key** to use their models programmatically:
  - Go to [https://platform.openai.com/](https://platform.openai.com/) and sign up or log in.
  - Create an API key from your [API keys dashboard](https://platform.openai.com/account/api-keys).
  - ⚠️ **Keep your API key private** and **do not** share or hardcode it in public notebooks.
- Note that **usage of the OpenAI API is not free**. You will need to:
  - Add a payment method.
  - Monitor your usage to avoid unexpected charges.
  - Optionally set usage limits from your account settings.
- You can refer to the **course’s Study Resources** for a step-by-step guide on creating an OpenAI account and retrieving your API key.

Then:
- Initialize the client with `OpenAI(api_key="YOUR_KEY_HERE")`.
- Send a test request using `.responses.create()` and the `"gpt-4o-mini"` model with a simple prompt:

  ```python
  response = client.responses.create(
      model="gpt-4o-mini",
      input="Write a one-sentence bedtime story about a unicorn."
  )
  print(response.output_text)


In [10]:
# CODE HERE
# Use as many coding cells as you need

load_dotenv()  # Loads environment variables from the .env file
api_key = os.getenv("KEY")  # Retrieve the value of the KEY variable

# If the API key is missing, stop execution and alert the user
if not api_key:
    raise ValueError("The variable 'KEY' was not found in the .env file")

# 2. Import the OpenAI client and initialize it
from openai import OpenAI

# Create an instance of the OpenAI client with the loaded API key
client = OpenAI(api_key=api_key)

# 3. Send a simple test request to confirm the connection works
response = client.responses.create(
    model="gpt-4o-mini",  # Small and fast GPT-4 model
    input="Hello! Can you confirm you're connected to the API, I'am Martin Jurado?"  # Test prompt
)

# 4. Print the model's output text
print(response.output_text)


Hello, Martin! I don’t have the ability to confirm connections like an API user would, but I'm here to help you with any questions or information you need. How can I assist you today?


## Retrieve Additional Metadata from Yahoo Finance

👉 **Instructions**:
- We will enrich our news dataset by retrieving **company-level metadata** using the `yfinance` library.
- The goal is to map each unique stock ticker (`TICKER`) in the dataset to:
  - `COMPANY_NAME`
  - `SECTOR`
  - `INDUSTRY`

> ℹ️ `yfinance` fetches live data from Yahoo Finance. If you're running this in a cloud environment or during peak hours, expect some tickers to fail or rate limits to apply.

✅ After this step, you will have a new DataFrame (e.g. `df_meta`) with the columns `TICKER`, `COMPANY_NAME`, `SECTOR`, `INDUSTRY` that maps tickers to their company names, sectors, and industries. This metadata will be useful later to add filters and analysis based on sector or industry categories.


In [11]:
# CODE HERE
# Use as many coding cells as you need
unique_tickers = df_news["TICKER"].unique()

# Prepare a list to hold metadata records
metadata_records = []

#Loop through each ticker and retrieve company metadata from Yahoo Finance
for ticker in unique_tickers:
    attempts = 0
    while attempts < 3:  # Retry up to 3 times
        try:
            yf_ticker = yf.Ticker(ticker)
            info = yf_ticker.info

            metadata_records.append({
                "TICKER": ticker,
                "COMPANY_NAME": info.get("longName", None),
                "SECTOR": info.get("sector", None),
                "INDUSTRY": info.get("industry", None)
            })
            break  # Exit retry loop if successful

        except Exception as e:
            attempts += 1
            print(f"⚠️ Attempt {attempts} failed for {ticker}: {e}")
            time.sleep(1)  # Wait before retrying

    time.sleep(0.5)  # Small pause between tickers to avoid rate limits



In [12]:
#Create a new DataFrame with the retrieved metadata
df_meta = pd.DataFrame(metadata_records, columns=["TICKER", "COMPANY_NAME", "SECTOR", "INDUSTRY"])

#Display the result
display(df_meta)

,TICKER,COMPANY_NAME,SECTOR,INDUSTRY
0,MMM,3M Company,Industrials,Conglomerates
1,AOS,A. O. Smith Corporation,Industrials,Specialty Industrial Machinery
2,ABT,Abbott Laboratories,Healthcare,Medical Devices
3,ABBV,AbbVie Inc.,Healthcare,Drug Manufacturers - General
4,ACN,Accenture plc,Technology,Information Technology Services
...,...,...,...,...
485,XEL,Xcel Energy Inc.,Utilities,Utilities - Regulated Electric
486,XYL,Xylem Inc.,Industrials,Specialty Industrial Machinery
487,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Restaurants
488,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Medical Devices


## Retrieval-Augmented Generation (RAG): Retrieve Documents and Generate Answers

👉 **Instructions**:

In this part of the assignment, your task is to build a simple Retrieval-Augmented Generation (RAG) pipeline that:

- Takes a user question as input.
- Searches the FAISS vector store to find a set of relevant financial news articles based on semantic similarity.
- Uses the retrieved news articles as context to generate a clear, concise answer to the question by interacting with the OpenAI language model.
- Returns both the generated answer and the underlying news snippets used for context.

### What you need to focus on:

- Implement a retrieval mechanism to query your vector store and obtain the top relevant documents for any question.
- Construct prompts that effectively combine retrieved news content with the user’s question to guide the language model’s response.
- Use the OpenAI API to generate answers grounded in the retrieved context.
- Organize the outputs so that for each question, you have:
  - The generated answer.
  - The collection of news excerpts used to produce that answer.

### What you will be provided:

- Helper functions to display outputs in markdown format.
- Lists of example questions covering topics, companies, and industries to test your implementation.

---

Your solution can take any form or structure you find appropriate, as long as it fulfills these core objectives. This exercise will give you hands-on experience with integrating retrieval and generation for practical applications in finance.


#### Print markdown
You can use the following function to print answers from GPT4o-mini in markdown.

In [13]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))

#### Predefined questions

In [14]:
questions_topic = [
"What are the major concerns expressed in financial news about inflation?",
"How is investor sentiment described in recent financial headlines?",
"What role is artificial intelligence playing in recent finance-related news stories?"
]

questions_company = [
"How is Microsoft being portrayed in news stories about artificial intelligence?",
"What financial news headlines connect Amazon with automation or logistics?"
]

questions_industry = [
"What are the main themes emerging in financial news about the semiconductor industry?",
"What trends are being reported in the retail industry?",
"What risks or challenges are discussed in recent news about the energy industry?"
]

In [15]:
# CODE HERE
# Use as many coding cells as you need
def retrieve_and_generate(question, k=5):
    """
    Retrieve top-k relevant news snippets from FAISS and use them as context
    to generate a concise answer using OpenAI GPT.
    """
    #Search for relevant documents using the FAISS index and model embeddings
    query_embedding = model.encode([question], convert_to_numpy=True)
    query_embedding = query_embedding / (np.linalg.norm(query_embedding, axis=1, keepdims=True) + 1e-10)  # Normalize
    
    scores, indices = faiss_index.search(query_embedding, k)
    
    retrieved_snippets = []
    for idx, score in zip(indices[0], scores[0]):
        meta = df_news.iloc[idx]
        snippet = meta['SUMMARY']
        retrieved_snippets.append((snippet, meta.to_dict(), score))
    
    # Build the context string with ticker, date, and summary for the prompt
    context_text = "\n\n".join(
        f"[{meta['TICKER']} | {meta['PUBLICATION_DATE']}]: {snippet}"
        for snippet, meta, score in retrieved_snippets
    )
    
    # Construct the prompt for the language model
    prompt = f"""
You are a financial news assistant.
Answer the following question based only on the context provided.

Question:
{question}

Context:
{context_text}

Answer:
"""
    #Generate answer using OpenAI API
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )
    
    return response.output_text, retrieved_snippets

# Question
question = "How is Microsoft being portrayed in news stories about artificial intelligence?"
answer, docs_used = retrieve_and_generate(question, k=5)

# Print the generated answer in markdown format
print_markdown(f"**Question:** {question}\n\n**Answer:** {answer}")

# Print the supporting news snippets with scores
print("\n--- Context Used ---")
for snippet, meta, score in docs_used:
    print(f"{meta['TICKER']} | {meta['PUBLICATION_DATE']} | Score: {score:.2f}")
    print(textwrap.fill(snippet, width=100))
    print()


**Question:** How is Microsoft being portrayed in news stories about artificial intelligence?

**Answer:** The context does not provide any specific information about how Microsoft is being portrayed in news stories about artificial intelligence. It mainly discusses other companies such as Meta Platforms and Salesforce in relation to AI. Therefore, I cannot determine Microsoft's portrayal from the provided information.


--- Context Used ---
META | 2025-05-31 | Score: 0.54
Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on
its legacy business.  Over the past two-and-a-half years, investors have heard about various
artificial intelligence (AI) investments that tech companies are making.

CRM | 2025-05-29 | Score: 0.51
D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce
(CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial
intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses
market share to competitors. Luria has the equivalent of a Sell rating on the stock. To watch more
expert insights and analysis on the latest market action, check out more Market Domination here.

JKHY | 2025-03-17 | Score: 0.48
We recently published a list of 12 AI News Investors Should Not Miss This Week. In this article, we
are going to take a look 

In [16]:
def run_analysis_and_reflection(questions, category_name):
    print(f"\n## Category: {category_name}\n")
    
    for i, question in enumerate(questions, start=1):
        answer, docs_used = retrieve_and_generate(question, k=5)
        
        print_markdown(f"### Question {i}: {question}")
        print_markdown(f"**Generated Answer:**\n{answer}\n")
        
        print("**Supporting News Snippets:**")
        for snippet, meta, score in docs_used:
            print(f"- [{meta['TICKER']} | {meta['PUBLICATION_DATE']}] (Score: {score:.2f}): {snippet[:200]}...")  # truncated
        
        print("\n**Reflection:**")
        print("- Relevance: ")
        print("- Completeness: ")
        print("- Bias or Noise: ")
        print("- Consistency: ")
        print("- Improvement Ideas: ")
        print("\n" + "-"*80 + "\n")
        time.sleep(21)
# Provided questions
run_analysis_and_reflection(questions_topic, "Topic")
run_analysis_and_reflection(questions_company, "Company")
run_analysis_and_reflection(questions_industry, "Industry")



## Category: Topic



### Question 1: What are the major concerns expressed in financial news about inflation?

**Generated Answer:**
The major concerns expressed in financial news about inflation include its persistence and the potential for an economic slowdown as highlighted by the Federal Reserve's May policy meeting.


**Supporting News Snippets:**
- [BLK | 2025-05-29] (Score: 0.58): The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown....
- [TSLA | 2025-05-31] (Score: 0.49): Key moments from the last seven days, plus a glimpse at the week ahead...
- [NVDA | 2025-05-31] (Score: 0.49): Key moments from the last seven days, plus a glimpse at the week ahead...
- [LULU | 2025-05-31] (Score: 0.49): Key moments from the last seven days, plus a glimpse at the week ahead...
- [AVGO | 2025-05-31] (Score: 0.49): Key moments from the last seven days, plus a glimpse at the week ahead...

**Reflection:**
- Relevance: 
- Completeness: 
- Bias or Noise: 
- Consistency: 
- Improvement Ideas: 

--------------------------------------------------------------------------------



### Question 2: How is investor sentiment described in recent financial headlines?

...

**Generated Answer:**
Current trends in the retail industry indicate a significant challenge, with stocks experiencing a downturn of approximately 13.7% over the past six months, which is worse than the S&P 500’s decline of 5.5%. Retailers are adapting their business models in response to technological changes in shopping behavior. Furthermore, there has been volatility in demand linked to fluctuations in consumer spending. Additionally, executives are adapting to legal changes regarding tariffs, adjusting supply chains, and implementing price increases that are already affecting consumer prices. Overall, the retail sector appears to be struggling against economic pressures and uncertain consumer behavior.


**Supporting News Snippets:**
- [KMX | 2025-05-12] (Score: 0.59): Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of consumer spending. This has stirred...
- [BBY | 2025-05-29] (Score: 0.51): As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increases already have hit shelves....
- [HLT | 2025-05-22] (Score: 0.50): The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand trends are working against their favor as the industry has tum...
- [PKG | 2025-04-26] (Score: 0.48): What trends should we look for it we want to identify stocks that can multiply in value over the long term? Amongst......
- [APD | 2025-05-14] (Score: 0.48): To find a multi-bagger stock, what are the underlying trends we should look for in a business

### Question 3: What risks or challenges are discussed in recent news about the energy industry?

**Generated Answer:**
Recent news about the energy industry highlights several risks and challenges:

1. **Potential Repeal of Subsidies**: Congress is advancing a bill that could eliminate key subsidies for renewable energy, potentially making projects uneconomical just as production ramps up.

2. **Oil Price Declines**: Companies are facing difficulties due to sliding oil prices, which may adversely affect their financial stability.

3. **Increased Tariffs**: Rising tariffs are adding financial pressure to energy companies.

4. **Shrinking Drilling Budgets**: As budgets for drilling diminish, companies are uncertain about future investments and operations.

These factors collectively pose significant risks to the future viability of both renewable and traditional energy sectors.


**Supporting News Snippets:**
- [NEE | 2025-05-23] (Score: 0.54): Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may bec...
- [ENPH | 2025-05-23] (Score: 0.54): Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped up production and renewable electricity generation in the U.S., those projects may bec...
- [HAL | 2025-05-21] (Score: 0.48): The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?...
- [BKR | 2025-05-21] (Score: 0.48): The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and AI demand offer enough support?...
- [FCX | 2025-05-21] (Score: 0.48): Copper is a critical metal for the U.S. industrial economy.

## Analysis & Questions - Section 1

### Analysis and Reflection on Retrieval and Generation Results
After running the RAG pipeline and obtaining answers along with their supporting news excerpts, take some time to carefully review both the generated responses and the retrieved contexts.

- **For each question, read the answer and then the corresponding news snippets used as context.**

- Reflect on the following points and document your observations:
1. **Relevance** 
2. **Completeness**  
3. **Bias or Noise** 
4. **Consistency**  
5. **Improvement Ideas**   

and answer the questions below:

#### **Question 1.** How well do the retrieved news snippets support the generated answer? Are the key facts or themes in the answer clearly grounded in the context?

The retrieved news snippets provide strong support for the generated answer. The key facts and themes—such as inflation concerns, economic slowdown risks, investor optimism tempered by skepticism, and the growing influence of artificial intelligence in finance—are explicitly present in the snippets. These excerpts offer diverse perspectives and examples that collectively validate the answer’s main points. The context clearly grounds the response in real, recent financial news, enhancing its credibility and relevance. However, the answer could be improved by explicitly referencing specific snippets or quotes, which would demonstrate even tighter integration between the source material and the generated summary.

#### **Question 2.** Does the answer fully address the question, or does it leave important aspects out? Consider if the retrieved context provided enough information to generate a thorough response.

The answer addresses the core of the question by describing investor sentiment as predominantly bullish while acknowledging the presence of skepticism due to potential institutional pressures on analysts. However, it could delve deeper into how this sentiment varies across different market sectors or investor groups, which may not be fully captured in the retrieved snippets. The context provides a solid foundation but is somewhat limited in scope, focusing mainly on price targets and analyst behavior. Therefore, while the response is accurate and informative, it leaves out a more nuanced exploration of investor sentiment dynamics that could enrich the answer.

#### **Question 3.** Are there any irrelevant or misleading snippets retrieved that may have influenced the answer? How might this affect the quality of the output?

Some snippets included general market updates or broader financial news that do not directly relate to investor sentiment, which might introduce slight noise into the context. While most retrieved snippets are relevant, the presence of less focused excerpts could dilute the precision of the answer or lead to generalized statements rather than specific insights. This can affect the quality by making the response less sharp or detailed, potentially overlooking subtler nuances in investor sentiment. Improving snippet filtering or weighting relevance more strictly could enhance the output’s clarity and accuracy.

#### **Question 4.**  Do the news snippets show consistent information, or are there conflicting viewpoints? How does the LLM handle potential contradictions in the context?

The news snippets generally show consistent information, portraying investor sentiment as cautiously optimistic with some skepticism due to potential institutional biases. However, minor variations exist in tone—some snippets emphasize bullish trends, while others highlight caution or risks. The LLM appears to synthesize these viewpoints by balancing optimism with caution, producing a nuanced answer that acknowledges both positive momentum and skepticism. This suggests the model effectively integrates potentially conflicting information by presenting a balanced summary rather than favoring one perspective exclusively.  

#### **Question 5.**  Based on your observations, suggest ways the retrieval or generation process could be improved (e.g., better filtering, adjusting `k`, refining prompt design).

The retrieval and generation process could be improved by:

* Implementing metadata-based filtering to focus on the most relevant and recent news snippets.
* Dynamically adjusting the number of retrieved documents (*k*) depending on the complexity of the question or the confidence of the retrieval.
* Optimizing the prompt design to give clearer instructions to the language model on how to handle conflicting information and prioritize key facts.
* Introducing a re-ranking step to reorder retrieved snippets by relevance before feeding them to the model.
* Managing the total context length to avoid exceeding model input limits, focusing on quality over quantity.
* Incorporating a feedback loop to iteratively refine retrieval and generation based on evaluation results.


## 🧠 Retrieval-Augmented Generation (RAG) v2: Adding Financial Metadata to Improve Generation

👉 **Instructions**:

In this part of the assignment, you’ll enhance your Retrieval-Augmented Generation (RAG) pipeline by incorporating *financial metadata* to provide more contextually rich answers.

Your goal is to evaluate whether metadata such as **company name**, **sector**, and **industry** helps the LLM generate **more accurate and grounded answers** to financial questions.

---

### ✅ What your updated pipeline should do:

- Retrieve relevant financial news articles using semantic similarity with FAISS.
- Enrich each retrieved document with financial metadata:
  - Ticker symbol
  - Full company name
  - Sector (e.g., Technology, Energy)
  - Industry (e.g., Semiconductors, Retail)
- Construct prompts that include both:
  - Retrieved news text
  - Associated metadata
- Send the prompt to the OpenAI model to generate an informed response.
- Return:
  - The final answer
  - The exact set of contextual documents used to produce that answer

---

### 🧪 Evaluation and Comparison:

You will test your improved RAG pipeline on the same three types of questions provided earlier:
- **Topic-focused** (e.g., inflation, interest rates)
- **Company-focused** (e.g., questions about Tesla, Nvidia)
- **Industry-focused** (e.g., semiconductors, utilities)


In [17]:
# CODE HERE
# Use as many coding cells as you need
# Function to retrieve top-k relevant documents with enriched financial metadata
def retrieve_documents_with_metadata(query, k=5):
    """
    Retrieve top-k relevant news articles along with financial metadata for each.
    """
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding /= np.linalg.norm(query_embedding, axis=1, keepdims=True) + 1e-10

    scores, indices = faiss_index.search(query_embedding, k)

    retrieved_docs = []
    for idx, score in zip(indices[0], scores[0]):
        news_row = df_news.iloc[idx]
        ticker = news_row['TICKER']
        meta_row = df_meta[df_meta['TICKER'] == ticker].iloc[0]

        doc = {
            'text': f"{news_row['TITLE']} : {news_row['SUMMARY']}",
            'ticker': ticker,
            'company_name': meta_row['COMPANY_NAME'],
            'sector': meta_row['SECTOR'],
            'industry': meta_row['INDUSTRY'],
            'publication_date': news_row['PUBLICATION_DATE'],
            'score': float(score)
        }
        retrieved_docs.append(doc)

    return retrieved_docs


In [18]:
# Function to build a prompt combining question, news text, and metadata
def build_prompt_with_financial_metadata(question, retrieved_docs):
    """
    Construct a prompt that provides question, contextual news, and financial metadata.
    """
    context_sections = []
    for doc in retrieved_docs:
        section = (
            f"[{doc['ticker']} | {doc['company_name']} | Sector: {doc['sector']} | "
            f"Industry: {doc['industry']} | Date: {doc['publication_date']}]\n"
            f"{doc['text']}"
        )
        context_sections.append(section)

    context_str = "\n\n---\n\n".join(context_sections)

    prompt = f"""
You are a financial analyst assistant.
Answer the question below based ONLY on the following financial news excerpts and their metadata.

Question:
{question}

Context:
{context_str}

Answer:
"""
    return prompt.strip()


In [19]:
#Full RAG pipeline function that retrieves, builds prompt, queries LLM, and returns answer + context
def rag_pipeline_with_metadata(question, k=5):
    """
    Runs the full RAG pipeline enhanced with financial metadata.
    """
    retrieved_docs = retrieve_documents_with_metadata(question, k)
    prompt = build_prompt_with_financial_metadata(question, retrieved_docs)

    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt
    )
    answer = response.output_text
    return answer, retrieved_docs


In [20]:
question = "How is Microsoft being portrayed in news stories about artificial intelligence?"
answer, docs_used = rag_pipeline_with_metadata(question, k=5)

print_markdown(f"**Question:** {question}\n\n**Answer:** {answer}")

print("\n--- Context Used ---")
for doc in docs_used:
    print(f"{doc['ticker']} | {doc['company_name']} | Sector: {doc['sector']} | Industry: {doc['industry']} | Date: {doc['publication_date']} | Score: {doc['score']:.2f}")
    print(doc['text'])
    print()


**Question:** How is Microsoft being portrayed in news stories about artificial intelligence?

**Answer:** The excerpts provided do not explicitly mention Microsoft or its portrayal in news stories about artificial intelligence. They focus mainly on other tech companies like Meta Platforms and Salesforce. Therefore, based on the available information, Microsoft is not directly addressed or described in terms of its role or perception in relation to artificial intelligence in the provided excerpts.


--- Context Used ---
META | Meta Platforms, Inc. | Sector: Communication Services | Industry: Internet Content & Information | Date: 2025-05-31 | Score: 0.54
This "Magnificent Seven" Stock Is Set to Skyrocket If Its AI Investments Pay Off : Meta Platforms has investments in several AI applications.  The tech giant's stock is only valued on its legacy business.  Over the past two-and-a-half years, investors have heard about various artificial intelligence (AI) investments that tech companies are making.

CRM | Salesforce, Inc. | Sector: Technology | Industry: Software - Application | Date: 2025-05-29 | Score: 0.51
How Salesforce has 'overcorrected' by leaning into AI : D.A. Davidson head of technology research Gil Luria joins Market Domination to discuss Salesforce (CRM) earnings and the company's trajectory. Luria says Salesforce is "too focused" on artificial intelligence (AI), as the other parts of its business "rapidly" decelerate and the company loses market share to competitors. 

In [22]:
def run_rag_on_questions(questions, category_name):
    print_markdown(f"## Category: {category_name}")
    for i, question in enumerate(questions, start=1):
        while True:
            try:
                answer, docs_used = rag_pipeline_with_metadata(question, k=5)
                break
            except httpx.HTTPStatusError as e:
                # Check if it's a 429 rate limit error
                if e.response.status_code == 429:
                    print("Rate limit reached. Waiting 20 seconds before retrying...")
                    time.sleep(20)
                else:
                    raise
        print_markdown(f"### Question {i}: {question}")
        print_markdown(f"**Answer:**\n{answer}\n")
        print("**Contextual News Snippets with Metadata:**")
        for doc in docs_used:
            print(f"- [{doc['ticker']} | {doc['company_name']} | Sector: {doc['sector']} | Industry: {doc['industry']} | Date: {doc['publication_date']}] (Score: {doc['score']:.2f})")
            print(f"  {doc['text'][:200]}...")  # Truncated snippet
        print("\n" + "-"*80 + "\n")

# Run for all three question categories
run_rag_on_questions(questions_topic, "Topic-Focused")
run_rag_on_questions(questions_company, "Company-Focused")
run_rag_on_questions(questions_industry, "Industry-Focused")

## Category: Topic-Focused

### Question 1: What are the major concerns expressed in financial news about inflation?

**Answer:**
The major concerns expressed in the financial news about inflation include:

1. **Persistent US Inflation**: There is a mounting concern from the Federal Reserve regarding ongoing inflationary pressures, which could potentially lead to an economic slowdown.

2. **Food Inflation**: Increased food prices are dampening hopes for a rate cut, indicating that inflation in essential goods is impacting economic forecasts and monetary policy decisions.

3. **Economic Impact**: These inflationary pressures, particularly in food, suggest broader implications for both consumer spending and economic growth, as persistent inflation could hinder the central bank's ability to adjust interest rates in response to changing economic conditions.


**Contextual News Snippets with Metadata:**
- [BLK | BlackRock, Inc. | Sector: Financial Services | Industry: Asset Management | Date: 2025-05-29] (Score: 0.58)
  Bitcoin price slips as Fed minutes flag US inflation risks : The Federal Reserve’s May policy meeting revealed mounting concern over persistent US inflation and the potential for economic slowdown....
- [TSLA | Tesla, Inc. | Sector: Consumer Cyclical | Industry: Auto Manufacturers | Date: 2025-05-31] (Score: 0.49)
  The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
- [NVDA | NVIDIA Corporation | Sector: Technology | Industry: Semiconductors | Date: 2025-05-31] (Score: 0.49)
  The Weekend: Food inflation dampens hopes of a rate cut as tariff twists and turns continue : Key moments from the last seven days, plus a glimpse at the week ahead...
- [LULU | lululemon athletica inc. | Sector: Consumer Cyclical | Indus

### Question 2: How is investor sentiment described in recent financial headlines?

...

**Answer:**
The retail industry is experiencing a notable decline, with retail stocks dropping by 13.7% over the past six months, significantly worse than the S&P 500's 5.5% loss. This downturn is attributed to volatile consumer demand and economic cycles affecting discretionary spending. Additionally, retailers are adapting by modifying their supply chains in response to challenges such as tariff policies, indicating an ongoing shift in business models due to technological advancements and external economic factors.


**Contextual News Snippets with Metadata:**
- [KMX | CarMax, Inc. | Sector: Consumer Cyclical | Industry: Auto & Truck Dealerships | Date: 2025-05-12] (Score: 0.59)
  3 Consumer Stocks That Concern Us : Retailers are adapting their business models as technology changes how people shop. Still, demand can be volatile as the industry is exposed to the ups and downs of...
- [BBY | Best Buy Co., Inc. | Sector: Consumer Cyclical | Industry: Specialty Retail | Date: 2025-05-29] (Score: 0.51)
  Retailers, Ducking Trade-War Curveballs, Stick to Their Plans : As legal rulings roll in on Trump’s tariff policies, retail executives say they have shifted their supply chains and many price increase...
- [HLT | Hilton Worldwide Holdings Inc. | Sector: Consumer Cyclical | Industry: Lodging | Date: 2025-05-22] (Score: 0.50)
  3 Consumer Stocks Skating on Thin Ice : The performance of consumer discretionary businesses is closely linked to economic cycles. Over the past six months, it seems like demand tr

### Question 3: What risks or challenges are discussed in recent news about the energy industry?

**Answer:**
Recent news about the energy industry highlights several significant risks and challenges:

1. **Legislative Risks for Renewable Energy**: A proposed bill in Congress threatens to repeal crucial subsidies for the renewable energy sector, which could render many newly ramped-up production projects uneconomical. This legislative move poses an existential threat to the industry, leading to crashing stock values for renewable energy companies.

2. **Volatility in Oil Prices**: The oil sector faces uncertainty as oil prices decline, which can impact profitability and operational budgets. Companies in oilfield services, like Halliburton and Baker Hughes, are particularly vulnerable to these price fluctuations.

3. **Rising Tariffs**: Increasing tariffs are affecting operational costs and competitiveness within the oil and gas equipment sector. This adds another layer of financial strain on companies that are already dealing with shrinking drilling budgets.

4. **Uncertain Demand**: While there is potential support from LNG (liquefied natural gas) and AI (artificial intelligence) demand, the overall future outlook remains uncertain for companies in the oilfield service market given the combination of falling prices and rising costs.

These challenges collectively indicate a turbulent environment for the energy sector, particularly for renewable sources and traditional oil and gas operations.


**Contextual News Snippets with Metadata:**
- [NEE | NextEra Energy, Inc. | Sector: Utilities | Industry: Utilities - Regulated Electric | Date: 2025-05-23] (Score: 0.54)
  Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
- [ENPH | Enphase Energy, Inc. | Sector: Technology | Industry: Solar | Date: 2025-05-23] (Score: 0.54)
  Renewable Energy Stocks Crash as U.S. Advances Bill That Could Decimate the Industry : Congress is pushing forward a bill that could upend the renewable energy industry.  Just as companies have ramped...
- [HAL | Halliburton Company | Sector: Energy | Industry: Oil & Gas Equipment & Services | Date: 2025-05-21] (Score: 0.48)
  Tariffs, Prices, and Pain: What's Next for Oilfield Service? : The likes of SLB, HAL and BKR face a tough future as oil prices slide, tariffs rise and drilling budgets shrink - can LNG and

## Analysis & Questions - Section 2

### Instructions: Evaluate Answers With and Without Metadata

For each question, compare the two answers provided:
- One generated **without** metadata
- One generated **with** metadata

---

### Steps:

1. Use the following evaluation criteria:
   - Clarity
   - Detail & Depth
   - Use of Context
   - Accuracy & Grounding
   - Relevance
   - Narrrative Flow

2. For each criterion, write brief notes comparing how the answer **without metadata** performs versus the answer **with metadata**.

3. Summarize your evaluation in a markdown table with the following columns:

| Criteria       | WITHOUT METADATA            | WITH METADATA             |
|----------------|----------------------------|--------------------------|
| Clarity        | [Your brief note here]     | [Your brief note here]   |
| Detail & Depth         | [Your brief note here]     | [Your brief note here]   |
| Use of Context        | [Your brief note here]     | [Your brief note here]   |
| Accuracy & Grounding       | [Your brief note here]     | [Your brief note here]   |
| Relevance      | [Your brief note here]     | [Your brief note here]   |
| Narrative Flow      | [Your brief note here]     | [Your brief note here]   |

---

**Note:** Keep comments short and clear for easy comparison.



### General comparison

| Criteria           | WITHOUT METADATA                                                                                       | WITH METADATA                                                                                                     |
|--------------------|------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| Clarity            | Answers are clear but limited to ticker and date context; can cause confusion about source.           | Much clearer by showing company name, sector, industry, and date; helps understand the context supporting the answer. |
| Detail & Depth     | Answers contain basic information but lack the rich context that could add depth.                     | Answers keep the same content but are enriched with context that adds depth and better justification.              |
| Use of Context     | Poor context; only snippets with ticker and date, no info to assess relevance or bias.                | Enriched context with metadata allows better assessment of snippet relevance and source origin.                    |
| Accuracy & Grounding| Answers are accurate per snippets but lack the ability to verify exactness or biases.                 | Greater grounding by clearly identifying source and sector, increasing confidence in accuracy.                     |
| Relevance          | Answers are relevant but less transparent on how each snippet contributes to the answer.              | Relevance is easier to validate thanks to transparency of origin and characteristics of retrieved content.         |
| Narrative Flow     | Adequate flow, though limited by the lack of enriched context.                                       | Better narrative flow as detailed context helps integrate information more cohesively.                             |


---

### Question 1: What are the major concerns expressed in financial news about inflation?

| Criteria                 | WITHOUT METADATA                                                                                                  | WITH METADATA                                                                                                 |
| ------------------------ | ----------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------- |
| **Clarity**              | Clear but very brief and generic: mentions "persistent US inflation" and "economic slowdown" without elaboration. | Much clearer, explicitly breaks down inflation concerns into persistence, food inflation, and tariff impacts. |
| **Detail & Depth**       | Lacks detail beyond naming two broad concerns.                                                                    | Adds depth by explaining how food inflation affects rate cut hopes and how tariffs complicate outlook.        |
| **Use of Context**       | Includes news snippets but doesn't explicitly relate them to the answer content.                                  | Integrates news snippets with metadata that directly support each concern, improving context and evidence.    |
| **Accuracy & Grounding** | Accurate but unsupported by explicit source citations or company/sector context.                                  | Well-grounded with dates, company names, sectors, and news headlines supporting each key point.               |
| **Relevance**            | Focuses on inflation broadly but omits some nuances (e.g., food inflation).                                       | More relevant and comprehensive by covering specific inflation factors and their economic implications.       |
| **Narrative Flow**       | Mostly a simple list of concerns with minimal transitions.                                                        | Smooth flow connecting persistent inflation concerns to economic consequences, creating a cohesive narrative. |

---

### Question 2: How is investor sentiment described in recent financial headlines?

| Criteria                 | WITHOUT METADATA                                                                                        | WITH METADATA                                                                                                      |
| ------------------------ | ------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------ |
| **Clarity**              | Fairly clear but somewhat generic: bullish sentiment and cautious notes are stated without elaboration. | Clearer explanation of optimism tempered by skepticism due to institutional pressures, with examples.              |
| **Detail & Depth**       | Mentions bullishness and skepticism but doesn’t elaborate on why or how this manifests in headlines.    | Explains analyst behavior, rare bearish ratings, and gives concrete context about market dynamics.                 |
| **Use of Context**       | News snippets provided but not explicitly tied to answer elements.                                      | Uses metadata news snippets to reinforce points about analyst optimism, skepticism, and bearish calls.             |
| **Accuracy & Grounding** | Generally accurate but without explicit source attribution or dates.                                    | Strongly grounded with dates, company info, and headline summaries supporting each sentiment aspect.               |
| **Relevance**            | Relevant but misses nuance on the pressure analysts face and rarity of bearish ratings.                 | Highly relevant with added insight into market psychology and investor caution.                                    |
| **Narrative Flow**       | Basic flow presenting optimism then caution; somewhat flat.                                             | More engaging narrative explaining why optimism exists but why skepticism is warranted, with smoother transitions. |

---

### Question 3: What role is artificial intelligence playing in recent finance-related news stories?

| Criteria                 | WITHOUT METADATA                                                                      | WITH METADATA                                                                                                 |
| ------------------------ | ------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------- |
| **Clarity**              | Clear overview of AI’s role but a bit high-level and generic.                         | Clear and specific examples of AI use in lending, investment, stock performance, and generative AI.           |
| **Detail & Depth**       | Basic summary of AI’s impact, lacking detailed company-specific examples or outcomes. | Provides multiple company-specific cases (Jack Henry, Meta, Palantir, Netflix) illustrating AI’s impact.      |
| **Use of Context**       | Supporting snippets are shown but not explicitly referenced.                          | Explicitly integrates metadata snippets tied to specific claims about AI, enhancing credibility.              |
| **Accuracy & Grounding** | Accurate but no direct citations or evidence linking answer to sources.               | Well-grounded with dates, sectors, companies, and quotes to back AI-related statements.                       |
| **Relevance**            | Relevant but a bit general and missing the financial performance angle.               | More relevant with connections to investor interest and stock market performance driven by AI.                |
| **Narrative Flow**       | Lists AI impacts without much narrative cohesion.                                     | Narratively cohesive with a story of how AI drives innovation, investment, and business process improvements. |

---

### Company Category — Question 1: How is Microsoft being portrayed in news stories about artificial intelligence?

| Criteria                 | WITHOUT METADATA                                        | WITH METADATA                                                                                    |
| ------------------------ | ------------------------------------------------------- | ------------------------------------------------------------------------------------------------ |
| **Clarity**              | States inability to answer due to lack of info clearly. | Same clear explanation that no relevant info about Microsoft AI portrayal was found in the data. |
| **Detail & Depth**       | Limited but appropriate given lack of data.             | Appropriately concise given no relevant context is present.                                      |
| **Use of Context**       | None, as no relevant info to analyze.                   | Context snippets confirm absence of Microsoft references.                                        |
| **Accuracy & Grounding** | Correctly avoids speculation without data.              | Properly grounded by the absence of evidence.                                                    |
| **Relevance**            | Relevant given question constraints.                    | Relevant and honest about data limitations.                                                      |
| **Narrative Flow**       | Straightforward and to the point.                       | Equally straightforward, no extra narrative needed due to data limitations.                      |

---

### Company Category — Question 2: What financial news headlines connect Amazon with automation or logistics?

| Criteria                 | WITHOUT METADATA                                                                               | WITH METADATA                                                                                                         |
| ------------------------ | ---------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------- |
| **Clarity**              | Clear but slightly vague: connects Amazon to AI generally without specific logistics examples. | Clearer with explicit references to AWS CEO statements on AI and indirect ties to logistics automation.               |
| **Detail & Depth**       | Basic answer with limited detail on automation or logistics.                                   | Adds nuance on headline contents, CEO quotes, and how AI relates to automation and logistics in operations.           |
| **Use of Context**       | Snippets shown but not explicitly tied to answer details.                                      | Metadata clearly referenced, highlighting dates, companies, and specific headlines connecting Amazon with automation. |
| **Accuracy & Grounding** | Correct but not strongly sourced or detailed.                                                  | Strongly grounded with source details reinforcing answer credibility.                                                 |
| **Relevance**            | Relevant but missing concrete logistics automation references.                                 | More relevant with clear evidence linking Amazon’s AI roadmap to logistics automation.                                |
| **Narrative Flow**       | Functional but somewhat list-like.                                                             | Better flow connecting the dots between AI, automation, and Amazon logistics strategy.                                |

---

### Industry Category — Question 1: What are the main themes emerging in financial news about the semiconductor industry?

| Criteria                 | WITHOUT METADATA                                                                      | WITH METADATA                                                                                     |
| ------------------------ | ------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------- |
| **Clarity**              | Clear listing of themes like revenue, investor sentiment, earnings, growth potential. | Same clarity but with more detailed explanation and specific company mentions and dates.          |
| **Detail & Depth**       | Basic but covers multiple themes broadly.                                             | Deeper coverage including concrete company examples and investment contexts.                      |
| **Use of Context**       | Context snippets included but not explicitly tied to each theme.                      | Strong contextual integration with metadata reinforcing points with company and sector specifics. |
| **Accuracy & Grounding** | Accurate but lacks citations linking back to snippets explicitly.                     | Well-grounded with references to companies, dates, and detailed snippet info.                     |
| **Relevance**            | Relevant to semiconductor themes but somewhat generic.                                | Highly relevant and specific to semiconductor industry trends and investor behavior.              |
| **Narrative Flow**       | Basic flow with list structure.                                                       | More narrative coherence tying themes together and showing relationships between them.            |

---

### Industry Category — Question 2: What trends are being reported in the retail industry?

| Criteria                 | WITHOUT METADATA                                                                   | WITH METADATA                                                                        |
| ------------------------ | ---------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **Clarity**              | Clear presentation of key trends like technology adaptation and demand volatility. | Clear with more detailed elaboration on each trend and its implications.             |
| **Detail & Depth**       | Covers main trends but briefly and somewhat superficially.                         | Greater detail including stock performance data and trade policy effects.            |
| **Use of Context**       | Context snippets present but not explicitly linked to claims.                      | Explicitly references metadata snippets, improving source credibility.               |
| **Accuracy & Grounding** | Accurate overall but without specific citations or numbers.                        | Well-supported with concrete dates, stock performance stats, and company details.    |
| **Relevance**            | Relevant to retail but lacking in some specific details.                           | More relevant with complete coverage of economic, technological, and policy impacts. |
| **Narrative Flow**       | Basic and list-like.                                                               | Better flow with connections drawn between trends and their effects on the industry. |

---

### Industry Category — Question 3: What risks or challenges are discussed in recent news about the energy industry?

| Criteria                 | WITHOUT METADATA                                              | WITH METADATA                                                                                             |
| ------------------------ | ------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------- |
| **Clarity**              | Clear overview of legislative, market, and economic risks.    | Clear and more detailed explanation of specific legislative bills, company impacts, and economic factors. |
| **Detail & Depth**       | Summarizes risks but lacks company-level examples and nuance. | Deeper detail including specific company challenges and legislative context.                              |
| **Use of Context**       | News snippets included but not explicitly tied to points.     | Metadata references back each risk with dates, company names, and sectors for credibility.                |
| **Accuracy & Grounding** | Accurate but less evidence-linked.                            | Strong grounding with specific data points and snippet metadata.                                          |
| **Relevance**            | Relevant but misses some detail on industry impacts.          | More comprehensive and relevant by covering both renewable and fossil fuel sectors in detail.             |
| **Narrative Flow**       | Functional but somewhat fragmented list.                      | Better flow with logical sequencing of risks and challenges creating a coherent picture.                  |

---
